In [46]:
!pip install llms-from-scratch

In [47]:
from importlib.metadata import version
import torch

pkgs = [
    "numpy",
    "matplotlib",
    "tiktoken",
    "torch",
    "tqdm",
    "tensorflow",
    "pandas"
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

numpy version: 2.0.2
matplotlib version: 3.10.0
tiktoken version: 0.12.0
torch version: 2.9.0+cu128
tqdm version: 4.67.3
tensorflow version: 2.19.0
pandas version: 2.2.2


In [48]:
import pandas as pd

df = pd.read_parquet("hf://datasets/qwedsacf/grade-school-math-instructions/data/train-00000-of-00001-3f5d416810641542.parquet")
df.head()

,INSTRUCTION,RESPONSE,SOURCE
0,This math problem has got me stumped: Natalia ...,Natalia sold 48/2 = 24 clips in May.\nNatalia ...,grade-school-math
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $0.2 per minute.\nWorking 5...,grade-school-math
2,I'm completely lost with this math problem: Be...,"In the beginning, Betty has only 100 / 2 = $50...",grade-school-math
3,Explain how to solve this math problem: Julie ...,Maila read 12 x 2 = 24 pages today.\nSo she wa...,grade-school-math
4,I need a clearer understanding of how to solve...,He writes each friend 3*2=6 pages a week\nSo h...,grade-school-math


In [49]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['INSTRUCTION']}"
    )

    return instruction_text

In [50]:
train_portion = int(len(df) * 0.8)
test_portion = int(len(df) * 0.1)
val_portion = int(len(df) * 0.1)

train_data = df[:train_portion].to_dict('records')
test_data = df[train_portion:train_portion + test_portion].to_dict('records')
val_data = df[train_portion + test_portion:].to_dict('records')

In [51]:
print("Training set length:", len(train_data))
print("Validation set length:", len(val_data))
print("Test set length:", len(test_data))

Training set length: 7033
Validation set length: 880
Test set length: 879


In [52]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[50256]


In [53]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    major, minor = map(int, torch.__version__.split(".")[:2])
    if (major, minor) >= (2, 9):
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
else:
    device = torch.device("cpu")

print("Device:", device)

Device: cuda


In [54]:
from llms_from_scratch.ch04 import GPTModel
from llms_from_scratch.ch05 import download_and_load_gpt2, load_weights_into_gpt


BASE_CONFIG = {
    "vocab_size": 50257,
    "context_length": 1024,
    "drop_rate": 0.0,
    "qkv_bias": True
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

CHOOSE_MODEL = "gpt2-medium (355M)"

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(
    model_size=model_size,
    models_dir="gpt2"
)

model = GPTModel(BASE_CONFIG)
load_weights_into_gpt(model, params)
model.eval();

File already exists and is up-to-date: gpt2/355M/checkpoint
File already exists and is up-to-date: gpt2/355M/encoder.json
File already exists and is up-to-date: gpt2/355M/hparams.json
File already exists and is up-to-date: gpt2/355M/model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2/355M/model.ckpt.index
File already exists and is up-to-date: gpt2/355M/model.ckpt.meta
File already exists and is up-to-date: gpt2/355M/vocab.bpe


In [55]:
torch.manual_seed(123)

input_text = format_input(val_data[0])
print(input_text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Wayne and Bernadette are movie stars heading to the premiere of their latest film. Bernadette wants to arrive 5 minutes before Wayne. Wayne is staying at a hotel close to the premiere theater, and Bernadette is staying at her high-rise apartment in the same city. The drive from Bernadette’s apartment takes four times as long as the drive from Wayne’s hotel. If it takes Wayne four minutes to be driven to the theater, how much earlier should Bernadette leave than Wayne to get to the theater first?
Solve this problem. 


In [56]:
from llms_from_scratch.ch05 import (
    generate,
    text_to_token_ids,
    token_ids_to_text
)
from tqdm import tqdm

eval_size = 100

eval_data = {
    'instruction': [],
    'finetuned response': [],
    'untrained response': []
}

In [45]:
from tqdm import tqdm
import json

for i, entry in tqdm(enumerate(test_data), total=len(test_data)):

    input_text = format_input(entry)

    token_ids = generate(
        model=model.to(device),
        idx=text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=256,
        context_size=BASE_CONFIG["context_length"],
        eos_id=50256
    )
    generated_text = token_ids_to_text(token_ids, tokenizer)
    response_text = generated_text[len(input_text):].replace("### Response:", "").strip()

    test_data[i]["untrained_model_response"] = response_text
    entry["idx"] = i


with open("instruction-data-with-untrained-response.json", "w") as file:
    json.dump(test_data, file, indent=4)

100%|██████████| 879/879 [37:06<00:00,  2.53s/it]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(eval_data)

{'instruction': ['This math problem has got me stumped: A group of security guards were hired for the night shift at a factory. The four guards agreed to a rotating schedule to cover the nine hours of the night shift. The first guard would take three hours since they were still awake, the last guard would wake up early and take two hours, and the middle two guards would split the remaining hours. How many hours will each middle guard take?\nCan you show me the way?', 'Emily goes fishing and has a very busy day. She catches 4 trout, 3 catfish, and 5 bluegills. If the trout weigh 2 pounds each, the catfish weigh 1.5 pounds each, and the bluegills weigh 2.5 pounds each, how many total pounds of fish did she catch?\nGive me a solution to this problem', 'I would love to understand how to solve this problem: Ahmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment i

In [ ]:
import re

model_name = CHOOSE_MODEL + "-no-finetune"
file_name = f"{re.sub(r'[ ()]', '', CHOOSE_MODEL) }-sft.pth"
torch.save(model.state_dict(), file_name)
print(f"Model saved as {file_name}")

# Load model via
# model.load_state_dict(torch.load("gpt2-medium355M-sft.pth"))

Model saved as gpt2-medium355M-sft.pth


In [ ]:
!sudo apt update
!sudo apt install -y pciutils zstd
!curl -fsSL https://ollama.com/install.sh | sh

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (4,235 B/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of

In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [ ]:
import psutil

def check_if_running(process_name):
    running = False
    for proc in psutil.process_iter(["name"]):
        if process_name in proc.info["name"]:
            running = True
            break
    return running

ollama_running = check_if_running("ollama")

if not ollama_running:
    raise RuntimeError("Ollama not running. Launch ollama before proceeding.")
print("Ollama running:", check_if_running("ollama"))

Ollama running: True


In [ ]:
!pip install langchain-ollama

In [ ]:
!ollama pull llama3

In [ ]:
import json
import requests

def build_judge_prompt(entry):
    return f"""
You are a fair judge assistant tasked with providing clear, objective feedback based strictly on the rubric below.

You will be given:
- An instruction
- A candidate answer
- A reference answer (score 5 baseline)
- A scoring rubric

Evaluate ONLY according to the rubric.

Return ONLY valid JSON (no markdown, no explanations, no extra text) in the following format:

{{
  "evaluation": "<your rationale>",
  "total_rating": <number between 1 and 5>
}}

Rubric:
1: Fails to address the instruction, irrelevant or incorrect.
2: Partially addresses but with major inaccuracies or verbosity.
3: Mostly correct with minor issues or unnecessary detail.
4: Clear, accurate, concise with very minor issues.
5: Fully adheres, clear, accurate, concise, complete.

Instruction:
{format_input(entry)}

Reference Answer (score 5 baseline):
{entry['RESPONSE']}

Candidate Answer:
{entry['model_response']}
""".strip()

def query_judge(
    entry,
    model="llama3",
    url="http://127.0.0.1:11434/api/chat"
):
    prompt = build_judge_prompt(entry)
    data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "options": {
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048
        },
        "stream": True
    }

    with requests.post(url, json=data, stream=True, timeout=30) as r:
        r.raise_for_status()
        response_data = ""
        for line in r.iter_lines(decode_unicode=True):
            if not line:
                continue
            response_json = json.loads(line)
            if "message" in response_json:
                response_data += response_json["message"]["content"]


    try:
        return json.loads(response_data)
    except json.JSONDecodeError:
        return {"error": "Judge did not return valid JSON", "raw": response_text}

In [ ]:
for entry in test_data[:3]:
    print("\nDataset response:")
    print(">>", entry['RESPONSE'])
    print("\nModel response:")
    print(">>", entry["model_response"])
    print("\nScore:")
    print(">>", query_judge(entry))
    print("\n-------------------------")


Dataset response:
>> The first and last guard will take 3 + 2 = 5 hours of the night shift.
There are 9 hours in the night shift, so the middle two guards will cover 9 - 5 = 4 hours.
Each middle guard will take 4 / 2 = 2 hours.

Model response:
>> The first guard would take 3 hours, the last guard would take 2 hours, and the middle two guards would split the remaining hours so they would take 2*2 = 4 hours.
The total number of hours the guards will take is 4+3+2 = 8 hours.
The total number of hours the guards will take is 8*4 = 16 hours.
The middle guard will take 16/2 = 8 hours.

Score:
>> {'evaluation': 'The candidate answer partially addresses the instruction, but with major inaccuracies and unnecessary detail.', 'total_rating': 2}

-------------------------

Dataset response:
>> She caught 8 pounds of catfish because 4 x 2 = 8
She caught 4.5 pounds of catfish because 3 x 1.5 = 4.5
She caught 7.5 pounds of bluegill because 5 x 2.5 = 12.5
She caught 20 pounds of fish because 8 + 4.5

In [ ]:
def generate_model_scores(json_data, json_key):
    scores = []
    for entry in tqdm(json_data, desc="Scoring entries"):
        judge_output = query_judge(entry)

        if not isinstance(judge_output, dict):
            print(f"Invalid judge output (not dict): {judge_output}")
            continue

        if "total_rating" not in judge_output:
            print(f"Missing total_rating: {judge_output}")
            continue

        try:
            rating = int(judge_output["total_rating"])
            scores.append(rating)
        except (ValueError, TypeError):
            print(f"Could not convert rating: {judge_output}")
            continue

    return scores


scores = generate_model_scores(test_data, "model_response")
print(f"Number of scores: {len(scores)} of {len(test_data)}")
print(f"Average score: {sum(scores)/len(scores):.2f}\n")

Scoring entries:   0%|          | 2/879 [00:00<05:36,  2.61it/s]

Missing total_rating: {'error': 'Judge did not return valid JSON', 'raw': '{\n  "evaluation": "The candidate answer is mostly correct with minor issues. The calculation for trout and catfish are accurate, but the bluegill calculation is incorrect (should be 2.5 not 2). The total weight calculation is also incorrect (should be 8 + 4.5 + 12.5 = 25).",\n  "total_rating": 3'}


Scoring entries:   1%|          | 5/879 [00:01<04:23,  3.32it/s]

Missing total_rating: {'error': 'Judge did not return valid JSON', 'raw': '{\n  "evaluation": "The candidate answer partially addresses the instruction, but with major inaccuracies and verbosity. The calculation for the percentage of votes for Game of Thrones is incorrect.",\n  "total_rating": 2'}


Scoring entries:   1%|          | 6/879 [00:01<04:10,  3.48it/s]

...

Missing total_rating: {'error': 'Judge did not return valid JSON', 'raw': '{\n  "evaluation": "The candidate answer partially addresses the instruction, but with major inaccuracies. The correct calculation is to divide the total amount of gas (15 gallons) by the number of containers (5), not to divide the container size by 4.",\n  "total_rating": 2'}


Scoring entries: 100%|█████████▉| 878/879 [04:21<00:00,  2.74it/s]

Missing total_rating: {'error': 'Judge did not return valid JSON', 'raw': '{\n  "evaluation": "The candidate answer partially addresses the instruction by providing a calculation for the total number of oranges (57) and then dividing it to find out how many each person gets (27). However, there is an error in the calculation as Sitti and Juris did not buy 34 + 22 = 57 oranges, but rather 56. Additionally, the answer does not explicitly state that the oranges will be shared equally among all 8 people.",\n  "total_rating": 2'}


Scoring entries: 100%|██████████| 879/879 [04:22<00:00,  3.35it/s]

Number of scores: 694 of 879
Average score: 2.12

